# Simulating saved results
This Jupyter notebook describes the case study for the the autonomous systems case study.

In [5]:
# Import libraries
from pacti.iocontract.utils import getVarlist
from PIL import Image
from pacti.terms.polyhedra import PolyhedralContract
import pdb
import random
import pickle as pkl
import numpy as np
import collections
from IPython.display import display, Image
from IPython.display import Latex
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

ModuleNotFoundError: No module named 'pacti.iocontract.utils'

## Part 1: Example

In this case study, we illustrate constructing lower bounds for elements of the confusion matrix. Consider the car pedestrian example examined in this case study. For the purposes of this jupyter notebook, the pedestrian is located 4 cells away from the car, which is starting at cell $C_1$ with an initial speed of 1 cell/step and a maximum possible speed of 2 cells/step.

In [6]:
Image("imglib/autonomy-stack.png")

NameError: name 'Image' is not defined

## Part 2: Getting probability points for ped, obj, empty:

Now, we illustrate the construction of controller contracts for the specific car pedestrain example given above. Suppose $c$ is the object class of the true environment. Since the probability of satisfaction of formula $\varphi_c$ is a nonlinear function of the elements of the confusion matrix, we compute an affine lower bound of this function via sampling. For each true positive rate, $\mathtt{TP}_{c}$, for a environment object type $c$, we generate 15 random instantiations of the false negative rates, $\mathtt{FN}_{(c',c)}$ and $\mathtt{FN}_{(c'',c)}$, where $c \notin \{c',c''\}$. For each instantiation of the confusion matrix, we construct the corresponding Markov chain $M(\mathtt{TP}_{c}, \mathtt{FN}_{(c',c)}, \mathtt{FN}_{(c'',c)})$ and compute the satisfaction probability accordingly. For all the probability samples, we then use linear programming to compute a lower bound as a function of the true positive rate.

In [4]:
# Here we load probability points from saved pickle files. To fully recompute the results, see recompute.ipynb

with open("lb.pkl", "rb") as f:
    lbounds_ped, points_ped, tpped_vals = pkl.load(f)
f.close()

with open("lbobj.pkl", "rb") as f:
    lbounds_obj, points_obj, tpobj_vals = pkl.load(f)
f.close()

with open("lbempty.pkl", "rb") as f:
    lbounds_emp, points_emp, tpemp_vals = pkl.load(f)
f.close()

NameError: name 'pkl' is not defined

In [3]:
# Plotting Probabilities:
def plot_probabilities_bounds(points, tp_vals, ubounds, lbounds, true_env):
    fig, ax = plt.subplots()
    ax.tick_params(axis='both', which='major', labelsize=10)
    plt.plot(tp_vals, points, 'b*',label='sampled')
    if ubounds != []:
        ub_m, ub_c = ubounds
        y_ub = ub_m*np.array(tp_vals) + ub_c
        plt.plot(tp_vals, y_ub, 'r')
        
    lb_m, lb_c = lbounds
    y_lb = lb_m*np.array(tp_vals) + lb_c
    plt.plot(tp_vals, y_lb, 'k',label='lower bound',linewidth=4)
    lb = min(tp_vals)
    ub = max(tp_vals)
    plt.legend(prop={'size': 10})
    
    plt.title('Lower bound of $\mathbb{P}_{'+true_env+'}$ as a function of $\mathtt{TP}_{'+true_env+'}$.')
    plt.ylabel('$\mathbb{P}_{'+true_env+'}$')
    plt.xlabel('$\mathtt{TP}_{'+true_env+'}$')

plot_probabilities_bounds(points_ped, tpped_vals, [], lbounds_ped, "ped")
plot_probabilities_bounds(points_obj, tpobj_vals, [], lbounds_obj, "obj")
plot_probabilities_bounds(points_emp, tpemp_vals, [], lbounds_emp, "emp")

NameError: name 'points_ped' is not defined

## Part 3: Controller Contracts

In [6]:
# First we construct each contract individually, and then merge the controller contract:
## Control contract for pedestrian class
def control_contract_ped(lbounds_ped):
    lb_m, lb_c = lbounds_ped
    mc= dict()
    mc = {"input_vars":["tp_ped"],
        "output_vars":[
            "P_ped"
        ],
        "assumptions":
        [
            {"coefficients":{"tp_ped":1},
            "constant":1},
            {"coefficients":{"tp_ped":-1},
            "constant":-0.6}
        ],
        "guarantees":
        [
          {"coefficients":{"P_ped":-1, "tp_ped":lb_m},
          "constant":-1*lb_c}
        ]}
    return mc

def control_contract_obj(lbounds_obj):
    lb_m, lb_c = lbounds_obj
    mc= dict()
    mc = {"input_vars":["tp_obj"],
        "output_vars":[
            "P_obj"
        ],
        "assumptions":
        [
            {"coefficients":{"tp_obj":1},
            "constant":1},
            {"coefficients":{"tp_obj":-1},
            "constant":-0.3}
        ],
        "guarantees":
        [
          {"coefficients":{"P_obj":-1, "tp_obj":lb_m},
          "constant":-1*lb_c}
        ]}
    return mc

def control_contract_emp(lbounds_emp):
    lb_m, lb_c = lbounds_emp
    mc= dict()
    mc = {"input_vars":["tp_emp"],
        "output_vars":[
            "P_emp"
        ],
        "assumptions":
        [
            {"coefficients":{"tp_emp":1},
            "constant":1},
            {"coefficients":{"tp_emp":-1},
            "constant":-0.6}
        ],
        "guarantees":
        [
          {"coefficients":{"P_emp":-1, "tp_emp":lb_m},
          "constant":-1*lb_c}
        ]}
    return mc

Cped = PolyhedralContract.from_dict(control_contract_ped(lbounds_ped))
Cobj = PolyhedralContract.from_dict(control_contract_obj(lbounds_obj))
Cemp = PolyhedralContract.from_dict(control_contract_emp(lbounds_emp))

print("Controller Contract for pedestrian class:\n" + str(Cped) + "\n")
print("Controller Contract for object class:\n" + str(Cobj)+ "\n")
print("Controller Contract for empty class:\n" + str(Cemp)+ "\n")

Controller Contract for pedestrian class:
InVars: [tp_ped]
OutVars:[P_ped]
A: [
  tp_ped <= 1.0
  -tp_ped <= -0.6
]
G: [
  -P_ped + 1.5801256015029996 tp_ped <= 0.6220524909016469
]

Controller Contract for object class:
InVars: [tp_obj]
OutVars:[P_obj]
A: [
  tp_obj <= 1.0
  -tp_obj <= -0.3
]
G: [
  -P_obj + 0.06834814849496082 tp_obj <= -0.9288940771532732
]

Controller Contract for empty class:
InVars: [tp_emp]
OutVars:[P_emp]
A: [
  tp_emp <= 1.0
  -tp_emp <= -0.6
]
G: [
  -P_emp + 0.20049999999999998 tp_emp <= -0.7996999999999667
]



In [7]:
# Construct merger of contracts:
C_controller = Cped.merge(Cobj) 
C_controller = C_controller.merge(Cemp)

print("Merged Controller Contract for all object classes:\n" + str(C_controller) + "\n")

Merged Controller Contract for all object classes:
InVars: [tp_ped, tp_obj, tp_emp]
OutVars:[P_ped, P_obj, P_emp]
A: [
  tp_ped <= 1.0
  -tp_ped <= -0.6
  tp_obj <= 1.0
  -tp_obj <= -0.3
  tp_emp <= 1.0
  -tp_emp <= -0.6
]
G: [
  -P_ped + 1.5801256015029996 tp_ped <= 0.6220524909016469
  -P_obj + 0.06834814849496082 tp_obj <= -0.9288940771532732
  -P_emp + 0.20049999999999998 tp_emp <= -0.7996999999999667
]



In [8]:
## System contract: P >= 0.99
def system_contract_dist():
    system_contract = {
        "input_vars":[
            "d"
        ],
        "output_vars":[
            "P_ped", "P_obj", "P_emp"
        ],
        "assumptions":
        [
            {"coefficients":{"d":-1}, "constant":0},
            {"coefficients":{"d":1}, "constant":10}
        ],
        "guarantees":
        [
            {"coefficients":{"P_ped":-1, "d":-0.099},
            "constant":-0.99},
            {"coefficients":{"P_obj":-1, "d":-0.08},
            "constant":-0.8},
            {"coefficients":{"P_emp":-1, "d":-0.095},
            "constant":-0.95},
        ]}
    return system_contract

Csys_dist = PolyhedralContract.from_dict(system_contract_dist())
print("System Contract (distance bounds):\n" + str(Csys_dist) + "\n")

System Contract (distance bounds):
InVars: [d]
OutVars:[P_ped, P_obj, P_emp]
A: [
  -d <= 0.0
  d <= 10.0
]
G: [
  -P_ped - 0.099 d <= -0.99
  -P_obj - 0.08 d <= -0.8
  -P_emp - 0.095 d <= -0.95
]



In [2]:
# Computing quotient contract:
C_dist = Csys_dist.quotient(C_controller)
print("Quotient Contract with distance bounds on true positive rates:\n" + str(C_dist)+"\n")

NameError: name 'Csys_dist' is not defined

## References